# Типы локов в PostgresQL
## Shared Lock (`SHARE`)
Другие транзакции могут читать, но не изменять данные пока держится блокировка.
## Exclusive Lock (`EXCLUSIVE`)
Другие транзакции не могут ни читать, ни изменять данные, пока держится блокировка.
## Row Lock (`FOR UPDATE`)
Ожидание блокировки на уровне строки. Применяется при обновлении или удалении строк.
## Advisory Lock
Программируемые блокировки, которые накладываются вручную с помощью `pg_advisory_lock`.

## Дедлок
Дедлок возникает когда две или более транзакций ожидают завершения друг друга, чтобы освободить ресурсы: строки и таблицы. Так создается цикл ожидания, называемый дедлоком.
**Пример:**
* Транзакция А держит блокировку на строке 1 и пытается получить блокировку на строке 2.
* Транзакция Б держит блокировку на строке 2 и пытается получить блокировку на строке 1.

### Безопасное завершение дедлока
#### 1. Завершение через логирование.
PostgreSQL может автоматически завершить один из процессов, если включена настройка `log_lock_waits` в `postgresql.conf` (/etc/postgresql/<version>/main/postgresql.conf или /var/lib/pgsql/data/postgresql.conf), либо через запрос в БД (для текущей сессии).

Вот настройка:

```bash
log_lock_waits = on
deadlock_timeout = 1s

#### 2. Обработка на уровне приложения
В SQLAlchemy except DeadlockDetected

In [ ]:
from sqlalchemy.exc import DeadlockDetected
from time import sleep

def safe_transaction(session):
    try:
        # выполняем транзакцию
        session.commit()
    except DeadlockDetected:
        # При дедлоке делаем паузу и повторяем транзакцию
        print("Deadlock detected, retrying...")
        session.rollback() # Откат, чтобы не допустить частичную запись
        sleep(1)  # Ожидаем и пытаемся снова
        safe_transaction() # Повторение